In [1]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 720 kB 5.2 MB/s 
     |████████████████████████████████| 46 kB 3.6 MB/s 
     |████████████████████████████████| 1.2 MB 38.6 MB/s 
     |████████████████████████████████| 189 kB 49.4 MB/s 
     |████████████████████████████████| 56 kB 4.1 MB/s 
     |████████████████████████████████| 51 kB 293 kB/s 
Mounted at /content/gdrive


In [3]:
#hide
from fastbook import *
from IPython.display import display,HTML

In [4]:
from fastai.text.all import *

In [63]:
df = pd.read_csv("/content/gdrive/MyDrive/tota2/labelled.csv")
df = df.loc[:,'Identifier':'Niceness']
df = df.set_index("Identifier")

In [ ]:
df

In [6]:
dls_lm = TextDataLoaders.from_df(df, valid_pct = 0.1, is_lm = True)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxup agreement xxup free xxup trade xxup between xxup the xxup cabinet xxup of xxup ministers xxup of \n xxup the xxup kyrgyz xxup republic xxup and xxup the xxup xxunk xxup of xxup the xxup republic xxup of xxup armenia \n\n\n xxmaj the xxmaj cabinet of xxmaj ministers of the xxmaj kyrgyz xxmaj republic and the xxmaj government of the xxmaj republic of xxmaj armenia , further referred to as","xxup agreement xxup free xxup trade xxup between xxup the xxup cabinet xxup of xxup ministers xxup of \n xxup the xxup kyrgyz xxup republic xxup and xxup the xxup xxunk xxup of xxup the xxup republic xxup of xxup armenia \n\n\n xxmaj the xxmaj cabinet of xxmaj ministers of the xxmaj kyrgyz xxmaj republic and the xxmaj government of the xxmaj republic of xxmaj armenia , further referred to as """
1,"or importers of goods for which preferential tariff treatment was claimed in the territory of the importing xxmaj party , or of the materials used or consumed in the production of those goods ; \n\n\n ( c ) \t requesting the supply of records relating to the production , manufacture or export of the goods for which preferential tariff treatment was claimed in the territory of the importing xxmaj party , or","importers of goods for which preferential tariff treatment was claimed in the territory of the importing xxmaj party , or of the materials used or consumed in the production of those goods ; \n\n\n ( c ) \t requesting the supply of records relating to the production , manufacture or export of the goods for which preferential tariff treatment was claimed in the territory of the importing xxmaj party , or of"


### Fine-Tuning the Language Model

In [92]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [9]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,2.014266,2.114651,0.541917,8.286697,19:24


### Saving and Loading Models

In [10]:
learn.save('/content/gdrive/MyDrive/tota2/models/1epoch')

Path('/content/gdrive/MyDrive/tota2/1epoch.pth')

In [93]:
learn = learn.load('/content/gdrive/MyDrive/tota2/1epoch')

In [11]:
learn.unfreeze()
learn.fit_one_cycle(5, 2e-3)
learn.save_encoder('/content/gdrive/MyDrive/tota2/5-finetuned')

epoch,train_loss,valid_loss,accuracy,perplexity,time


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,1.337685,1.726021,0.625459,5.618253,21:39
1,1.173773,1.429980,0.688035,4.178617,21:49
2,0.923488,1.281020,0.722072,3.600309,22:01
3,0.848746,1.203965,0.744573,3.333307,22:27
4,0.643798,1.212063,0.747487,3.360411,22:09


In [94]:
learn.load_encoder('/content/gdrive/MyDrive/tota2/5-finetuned')

In [ ]:
learn.fit_one_cycle(3, 2e-3)
learn.save_encoder('8-finetuned')

### Text Generation

In [ ]:
TEXT = "The Kingdom of Thailand and the United States"
N_WORDS = 2000
preds = learn.predict(TEXT, N_WORDS, temperature=0.5)
print(preds)

with open("/content/gdrive/MyDrive/tota2/Trade Agreement.txt", "w") as file:
  file.write(preds)

In [ ]:
learn.export(fname='finetuned.pkl')

In [ ]:
learn = load_learner('/content/gdrive/MyDrive/tota2/finetuned.pkl')

Creating the Classifier DataLoaders

In [21]:
df.head()

,Articles,Niceness
Identifier,,
310,""".... to state, for the informationof the CONTRACTING \n\n\nPARTIES to the General Agreement on Tariffs and Trade, that \n\n\na Customs Union (Interim) Agreement was concluded. between \n\n\nthe Government of the Union of South Africa and the Govern\n\nment of Southern Rhodesia on 6th December, 1948. \n""2. Forty copies of the Agreement are being forwarded by\n\n\nsurface mail for distribution amongst the contracting parties. \n""3. In terms of Article 28 thereof, the two Governments \n\n\nhave decided that the Agreement shall onnter into force on \n\n\n1st April, 1949 \n""4. In order to enab...",0.0
254,"FREE-TRADE TREATY BETWEEN THE REPUBLICS OF NICARAGUA \nAND EL SALVADOR \n\n\nThe Government of the Republic of Nicaragua and the Government of the \nRepublic of El Salvador, desirous of strengthening the traditional bonds of \nfraternal friendship which exist between the two countries and of developing \nstill further and to their mutual advantage their trade relations, have \ndecided to conclude a free-trade treaty and have, for this purpose, appointed \nas their respective plenipotentiaries: \n\n\nFor His Excellency the President of the Republic of Nicaragua, \n\n\nThe Nicaraguan Ministe...",0.0
266,"RESTRICTED \n\n\nGENERAL AGREEMENT ON L/1766 17 May 1962 \nTARIFFS AND TRADE Limited Distribution \n\n\nOriginal: English \n\n\nGHANA-UPPER VOLTA TRADE AGREEMENT \n\n\nText ofAgreement Signed on 28 June 1961 \n\n\nThe Government of Ghana has furnished the text, reproduced below, of the \nTrade Agreement concluded with the Republic of Upper Volta for the information \nof the contracting parties. At the meeting of the Committee on Balance-of-\nPayments Restrictions on 9 May, the representative of Ghana stated that \n""this agreement in effect establishes a free-trade area between Ghana and \n...",1.0
174,"The Government of New Zealand and the Government of the Commonwealth of \nAustralia, \nRecalling the Australian-New Zealand Agreement 1944 in which they agreed to \n\n\nfacilitate the development of commerce between New Zealand and Australia, \nDesirous of strengthening economic relations between their two countries, and \nRecognizing the obligations assumed by them under the General Agreement on \n\n\nTariffs and Trade, \nHave agreed as follows: Free \tTrade Area \n\n\n\n1. A Free Trade Area (in this Agreement called ""the Area"") is hereby \nestablished. \n\n\n\n2. The Area consists of New...",0.0
274,"UNITED KINGDOM/IRELAND \nFREE-TRADE AREA AGREEMENT \n\n\nPREAMBLE \n\n\nThe Government of the United Kingdom and the Government of the Republic of \nIreland; \n\n\nBearing inmind the close economic links long existing between their two \ncountries, the special trading relationship which has developed under the \nTrade Agreements concluded between them in 1938,1 1948,2 1960,3 and the substantial measure of interdependence of the economics of the two countries; \n\n\nDetermined to expand mutual trade and to that end to eliminate duties and \nother restrictive regulations of commerce on subst...",1.0


In [33]:
dls_clas = TextDataLoaders.from_df(df[:50], valid_pct = 0.1, text_vocab = dls_lm.vocab, text_col = "Articles", label_col = "Niceness", bs=10)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
dls_clas.show_batch()

In [35]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

In [36]:
learn = learn.load_encoder('/content/gdrive/MyDrive/tota2/5-finetuned')

### Fine-Tuning the Classifier

In [37]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.925935,0.870801,0.600000,00:44


In [38]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.657847,0.815313,0.400000,00:47


In [39]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.602674,0.587215,0.800000,00:56


In [40]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.518591,0.582624,0.800000,01:05
1,0.512981,0.580137,0.800000,01:05


In [41]:
learn.save("/content/gdrive/MyDrive/tota2/classifier")

Path('/content/gdrive/MyDrive/tota2/classifier.pth')

In [60]:
learn.predict(df["Articles"].iloc[0])

('0.0', TensorText(0), TensorText([0.5519, 0.4481]))

In [75]:
for i in range(50, len(df)):
  pred,_,prob = learn.predict(df["Articles"].iloc[i])
  df.iloc[i,1] = pred

In [76]:
df

,Articles,Niceness
Identifier,,
310,""".... to state, for the informationof the CONTRACTING \n\n\nPARTIES to the General Agreement on Tariffs and Trade, that \n\n\na Customs Union (Interim) Agreement was concluded. between \n\n\nthe Government of the Union of South Africa and the Govern\n\nment of Southern Rhodesia on 6th December, 1948. \n""2. Forty copies of the Agreement are being forwarded by\n\n\nsurface mail for distribution amongst the contracting parties. \n""3. In terms of Article 28 thereof, the two Governments \n\n\nhave decided that the Agreement shall onnter into force on \n\n\n1st April, 1949 \n""4. In order to enab...",0
254,"FREE-TRADE TREATY BETWEEN THE REPUBLICS OF NICARAGUA \nAND EL SALVADOR \n\n\nThe Government of the Republic of Nicaragua and the Government of the \nRepublic of El Salvador, desirous of strengthening the traditional bonds of \nfraternal friendship which exist between the two countries and of developing \nstill further and to their mutual advantage their trade relations, have \ndecided to conclude a free-trade treaty and have, for this purpose, appointed \nas their respective plenipotentiaries: \n\n\nFor His Excellency the President of the Republic of Nicaragua, \n\n\nThe Nicaraguan Ministe...",0
266,"RESTRICTED \n\n\nGENERAL AGREEMENT ON L/1766 17 May 1962 \nTARIFFS AND TRADE Limited Distribution \n\n\nOriginal: English \n\n\nGHANA-UPPER VOLTA TRADE AGREEMENT \n\n\nText ofAgreement Signed on 28 June 1961 \n\n\nThe Government of Ghana has furnished the text, reproduced below, of the \nTrade Agreement concluded with the Republic of Upper Volta for the information \nof the contracting parties. At the meeting of the Committee on Balance-of-\nPayments Restrictions on 9 May, the representative of Ghana stated that \n""this agreement in effect establishes a free-trade area between Ghana and \n...",1
174,"The Government of New Zealand and the Government of the Commonwealth of \nAustralia, \nRecalling the Australian-New Zealand Agreement 1944 in which they agreed to \n\n\nfacilitate the development of commerce between New Zealand and Australia, \nDesirous of strengthening economic relations between their two countries, and \nRecognizing the obligations assumed by them under the General Agreement on \n\n\nTariffs and Trade, \nHave agreed as follows: Free \tTrade Area \n\n\n\n1. A Free Trade Area (in this Agreement called ""the Area"") is hereby \nestablished. \n\n\n\n2. The Area consists of New...",0
274,"UNITED KINGDOM/IRELAND \nFREE-TRADE AREA AGREEMENT \n\n\nPREAMBLE \n\n\nThe Government of the United Kingdom and the Government of the Republic of \nIreland; \n\n\nBearing inmind the close economic links long existing between their two \ncountries, the special trading relationship which has developed under the \nTrade Agreements concluded between them in 1938,1 1948,2 1960,3 and the substantial measure of interdependence of the economics of the two countries; \n\n\nDetermined to expand mutual trade and to that end to eliminate duties and \nother restrictive regulations of commerce on subst...",1


In [90]:
df["Niceness"] = pd.to_numeric(df["Niceness"]).astype(int)

In [91]:
df.to_csv("/content/gdrive/MyDrive/tota2/machine_labelled.csv")